In [ ]:
## Qwen的分词是padding在右边的，注意哦

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
torch.manual_seed(114514)

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("./Qwen-VL-Chat", trust_remote_code=True)

# use bf16
model = AutoModelForCausalLM.from_pretrained("./Qwen-VL-Chat", device_map="auto", trust_remote_code=True, bf16=True).eval()

# Specify hyperparameters for generation
model.generation_config = GenerationConfig.from_pretrained("./Qwen-VL-Chat", trust_remote_code=True)



c:\Users\WuNein\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


In [3]:
tokenizer('yes no')

{'input_ids': [9693, 902], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}

In [5]:
query = tokenizer.from_list_format([
    {'image': './cat.png'}, # Either a local path or an url
    {'text': """what is this picture?"""},
])
from qwen_generation_utils import (
    HistoryType,
    make_context,
    decode_tokens,
    get_stop_words_ids,
    StopWordsLogitsProcessor,
)
raw_text, context_tokens = make_context(
    tokenizer,
    query,
    history=None,
    system="You are a helpful assistant.",
    max_window_size=model.generation_config.max_window_size,
    chat_format=model.generation_config.chat_format,
)

In [2]:
model.transformer.h.to('cpu')
torch.cuda.empty_cache()

In [6]:
with torch.no_grad():
    images = model.transformer.visual.encode(['./1/10.png','./1/11.png','./1/12.png'])
    assert images.shape[0] == len(images)

In [4]:
images.size()

torch.Size([3, 256, 4096])

self-alignment -> image embedding 与 他的Dense Text embedding (descrption mean in a word )

In [3]:
history = [
    (
        'Picture 1:<img>./s0004800.jpg</img>\nCaptions: [\n    "red and yellow train stopped at a station",\n    "A train is stopped at the train station.",\n    "A orange and red train parked at the station.",\n    "a parked train sits beside a building",\n    "A train sitting in a train station in a rural area."\n]\nShort negative examples that mean the exact opposite of what is described in Captions and picture (opposite meaning): ',
        '"a red and orange building sits beside a running train"',
    )
]

In [6]:
history = [
    (
        'Picture 1:<img>./cat.png</img>\nThis picture means in one word:',
        'cat',
    )
]

In [ ]:
# 1st dialogue turn
query = tokenizer.from_list_format([
    {'image': './data/flickr30_4/s0028289.jpg'}, # Either a local path or an url
    {'text': """Captions: [
        "Two men sitting on a bench on a street next to two women doing the same.",
        "Two people sit on a bench leaned against a building with writing on it.",
        "Many people standing and sitting on the street in front of billboards.",
        "People sitting on benches beside the street.",
        "Two people sit on a bench on a city street."
    ]
Short negative examples that mean the exact opposite to what is described in Captions and picture (opposite meaning): """},
])
response, history = model.chat(tokenizer, query=query, history=history)
print(response)
# 图中是一名女子在沙滩上和狗玩耍，旁边是一只拉布拉多犬，它们处于沙滩上。

Qwen-VL理解不了这个需求

In [4]:
from qwen_generation_utils import (
    HistoryType,
    make_context,
    decode_tokens,
    get_stop_words_ids,
    StopWordsLogitsProcessor,
)

In [5]:
history = [
    (
        'Picture 1:<img>./cat.png</img>\nThis picture means in one word:',
        'cat',
    )
]

In [2]:
query = tokenizer.from_list_format([
    # {'image': './cat.png'}, # Either a local path or an url
    # {'text': """This picture means in one word: cat"""},
    # {'image': './data/flickr30_4/s0028289.jpg'}, # Either a local path or an url
    {'text': """This picture means in one word:"""},
])

In [5]:
raw_text, context_tokens = make_context(
    tokenizer,
    query,
    history=None,
    system="You are a helpful assistant.",
    max_window_size=model.generation_config.max_window_size,
    chat_format=model.generation_config.chat_format,
)

In [21]:
with torch.no_grad():
    input_ids = torch.tensor([context_tokens]).to(model.device)
    output = model(input_ids, attention_mask = input_ids.ne(tokenizer.pad_token_id), output_hidden_states = True, return_dict=True)


In [13]:
output.hidden_states[-1][:, -1, :].size()

torch.Size([1, 4096])

In [22]:
predicted_token_id = output.logits[:,-1,:].argmax(dim=-1)
decoded_text = tokenizer.decode(predicted_token_id)
decoded_text


'ca'

In [1]:
import os
import json

folder_path = "./"
files = os.listdir(folder_path)

total = 0
counter = 0
# 遍历文件夹中的每个文件

big = []

for file in files:
    # 确保文件是以'.json'为扩展名的JSON文件
    if file.endswith(".json") and "flickr" in file:
        # 构建完整的文件路径
        file_path = os.path.join(folder_path, file)
        counter += 1
        # 打开文件并读取JSON数据
        with open(file_path, "r", encoding="utf-8") as json_file:
            # 尝试解析JSON数据
            data = json.load(json_file)
            big.extend(data)

merged_file = open('flickr30k_no3.json', 'w+', encoding='utf-8')

json.dump(big,merged_file,ensure_ascii=False)

merged_file.close()

In [3]:

texts = [
'A chocolate cake with lots of candles in it.',
 'Someone pinches a bite off of a homemade chocolate cake.',
 'The person is getting ready to serve the cake. ',
 'A person takes a piece of chocolate cake.',
 'a chocolate cake with with some candy on it ',
]# correct answer
texts.extend(
    [
         'a candy with some chocolate cake on top of it',
             "red and yellow train stopped at a station",
    "A train is stopped at the train station.",
    "A orange and red train parked at the station.",
    "a parked train sits beside a building",
    "A train sitting in a train station in a rural area.",
    ]
) # false answer


In [4]:
import os
from qwen_generation_utils import (
    HistoryType,
    make_context,
    decode_tokens,
    get_stop_words_ids,
    StopWordsLogitsProcessor,
)
text_outputs = []
for item in texts:
    query = tokenizer.from_list_format([
        {'text': f"""高等学校或研究机关中指导他人学习、进修、或撰写学术论文的教师或科研人员。\n上面这句话用一个中文词来表达是：导师。"""},
        {'text': f"""{item}\n上面这句话用一个中文词来表达是："""},
    ])
    raw_text, context_tokens = make_context(
        tokenizer,
        query,
        history=None,
        system="You are a helpful assistant.",
        max_window_size=model.generation_config.max_window_size,
        chat_format=model.generation_config.chat_format,
    )
    tokenizer.pad_token_id = tokenizer.eod_id

    with torch.no_grad():
        input_ids = torch.tensor([context_tokens]).to(model.device)
        my_text_output = model(input_ids, attention_mask = input_ids.ne(tokenizer.pad_token_id), output_hidden_states = True, return_dict=True)
        predicted_text_token_id = my_text_output.logits[:, -1, :].argmax(dim=-1)
        text_output = my_text_output.hidden_states[-1][:, -1, :]
    text_outputs.append(text_output)
temp_text_all = (torch.cat(text_outputs,dim=0).float().cpu())



In [21]:
emb_arr = []

In [24]:
file_path = "./pixel_pic/2.png"

query = tokenizer.from_list_format([
    {'image': './cat.png'}, # Either a local path or an url
    {'text': """上面这个图用一个中文词来表达是：猫。"""},
    {'image': file_path}, # Either a local path or an url
    {'text': """上面这个图用一个中文词来表达是："""},
])
raw_text, context_tokens = make_context(
    tokenizer,
    query,
    history=None,
    system="You are a helpful assistant.",
    max_window_size=model.generation_config.max_window_size,
    chat_format=model.generation_config.chat_format,
)
tokenizer.pad_token_id = tokenizer.eod_id
with torch.no_grad():
    input_ids = torch.tensor([context_tokens]).to(model.device)
    outputs = model(input_ids, attention_mask = input_ids.ne(tokenizer.pad_token_id), output_hidden_states = True, return_dict=True)
    predicted_token_id = outputs.logits[:, -1, :].argmax(dim=-1)
    image_outputs = outputs.hidden_states[-1][:, -1, :]

temp_image_all = image_outputs.float().cpu()
emb_arr.append(temp_image_all)

torch.Size([2, 256, 4096])


In [25]:
from scipy.spatial.distance import cosine
cosine_sim_0_1 = 1 - cosine(emb_arr[0][0], emb_arr[1][0])
cosine_sim_0_2 = 1 - cosine(emb_arr[0][0], emb_arr[2][0])

print(cosine_sim_0_1)
print(cosine_sim_0_2)

0.8933104276657104
0.8911939263343811


In [4]:
import torch.nn.functional as F

similarity_func = lambda s1, s2: torch.nan_to_num(
    F.cosine_similarity(torch.nan_to_num(s1), torch.nan_to_num(s2), dim=-1)
)

In [6]:
scores = similarity_func(temp_text_all, temp_image_all)
scores.tolist(), scores.argmax().item()

([0.5932328701019287,
  0.6057933568954468,
  0.5303196907043457,
  0.6028374433517456,
  0.6286076903343201,
  0.5483824014663696,
  0.34998637437820435,
  0.2931695580482483,
  0.3104252219200134,
  0.2342160940170288,
  0.2860998213291168],
 4)

In [2]:
import os
from qwen_generation_utils import (
    HistoryType,
    make_context,
    decode_tokens,
    get_stop_words_ids,
    StopWordsLogitsProcessor,
)

texts = [
    # "A chocolate cake with lots of candles in it.",
    #  'Someone pinches a bite off of a homemade chocolate cake.',
    #  'The person is getting ready to serve the cake. ',
    #  'A person takes a piece of chocolate cake.',
    #  'a chocolate cake with with some candy on it ',
]  # correct answer
texts.extend(
    [
        "a candy with some chocolate cake on top of it",
        "red and yellow train stopped at a station",
        "A train is stopped at the train station.",
        "A orange and red train parked at the station.",
        "a parked train sits beside a building",
        "A train sitting in a train station in a rural area.",
    ]
)  # false answer


file_path = "./s0004801.jpg"

query = tokenizer.from_list_format(
    [
        {"image": file_path[:-3] + "jpg"},  # Either a local path or an url
        {"text": """上面这个图与下面哪个选项中的内容关系最密切，如果都没有关系，请输出None："""},
    ]
    + [
        {"text": str(chr(ord("A") + index)) + ".\t" + item}
        for index, item in enumerate(texts)
    ] + [{"text": "None.\t" + "以上所有句子都不对"}]
)
raw_text, context_tokens = make_context(
    tokenizer,
    query,
    history=None,
    system="You are a helpful assistant.",
    max_window_size=model.generation_config.max_window_size,
    chat_format=model.generation_config.chat_format,
)
tokenizer.pad_token_id = tokenizer.eod_id
with torch.no_grad():
    input_ids = torch.tensor([context_tokens]).to(model.device)
    outputs = model(
        input_ids,
        attention_mask=input_ids.ne(tokenizer.pad_token_id),
        output_hidden_states=True,
        return_dict=True,
    )
    predicted_token_id = outputs.logits[:, -1, :]

In [16]:
import os
from qwen_generation_utils import (
    HistoryType,
    make_context,
    decode_tokens,
    get_stop_words_ids,
    StopWordsLogitsProcessor,
)

texts = [
    "A chocolate cake with lots of candles in it.",
     'Someone pinches a bite off of a homemade chocolate cake.',
    #  'The person is getting ready to serve the cake. ',
    #  'A person takes a piece of chocolate cake.',
    #  'a chocolate cake with with some candy on it ',
]  # correct answer
texts.extend(
    [
        "a candy with some chocolate cake on top of it",
        "red and yellow train stopped at a station",
        "A train is stopped at the train station.",
        "A orange and red train parked at the station.",
        "a parked train sits beside a building",
        "A train sitting in a train station in a rural area.",
    ]
)  # false answer

# texts = [
#     "red and yellow train stopped at a station",
#     "A train is stopped at the train station.",
#     "A orange and red train parked at the station.",
#     # "a parked train sits beside a building",
#     # "A train sitting in a train station in a rural area.",
# ] # correct answer
# texts.extend(
#     [
#         "a red and yellow platform stop beside the train",
#         "A red and yellow train is loading passengers at the station.",
#         "a red and yellow building sits beside a running train",
#     ]
# ) # false answer

file_path = "./s0004801.jpg"
query = tokenizer.from_list_format(
    [
        {"image": file_path[:-3] + "jpg"},  # Either a local path or an url
        {"text": """多项选择题：下面哪些选项正确的描述了这个图，如果都不对请输出None：\n"""}
        
    ]
    + [
        {"text": str(chr(ord("A") + index)) + ".\t" + item + "\n"}
        for index, item in enumerate(texts)
    ] 
    + [{"text": "None.\t" + "以上所有描述都不对\n"}]
    + [{"text": "答：AB\n\n"}]
    +
    [
        {"image": file_path[:-3] + "jpg"},  # Either a local path or an url
        {"text": """多项选择题：下面哪些选项正确的描述了这个图，如果都不对请输出None：\n"""}
        
    ]
    + [
        {"text": str(chr(ord("A") + index)) + ".\t" + item + "\n"}
        for index, item in enumerate(texts)
    ] 
    + [{"text": "None.\t" + "以上所有描述都不对\n"}]
    + [{"text": "答："}],
)
response, history = model.chat(tokenizer, query=query, history=None)
print(response)

A. A chocolate cake with lots of candles in it.


In [17]:
print(query) # ABC 正确

Picture 1:<img>./s0004801.jpg</img>
多项选择题：下面哪些选项正确的描述了这个图，如果都不对请输出None：
A.	A chocolate cake with lots of candles in it.
B.	Someone pinches a bite off of a homemade chocolate cake.
C.	a candy with some chocolate cake on top of it
D.	red and yellow train stopped at a station
E.	A train is stopped at the train station.
F.	A orange and red train parked at the station.
G.	a parked train sits beside a building
H.	A train sitting in a train station in a rural area.
None.	以上所有描述都不对
答：AB

Picture 2:<img>./s0004801.jpg</img>
多项选择题：下面哪些选项正确的描述了这个图，如果都不对请输出None：
A.	A chocolate cake with lots of candles in it.
B.	Someone pinches a bite off of a homemade chocolate cake.
C.	a candy with some chocolate cake on top of it
D.	red and yellow train stopped at a station
E.	A train is stopped at the train station.
F.	A orange and red train parked at the station.
G.	a parked train sits beside a building
H.	A train sitting in a train station in a rural area.
None.	以上所有描述都不对
答：


In [7]:
history = [
    (
        """Picture 1:<img>./s0004801.jpg</img>
多项选择题：下面哪些选项正确的描述了这个图的部分特征：
A.	A chocolate cake with lots of candles in it.
B.	Someone pinches a bite off of a homemade chocolate cake.
C.	a candy with some chocolate cake on top of it
答：""",
        'AB',
    )
]

query = """Picture 2:<img>./s0004800.jpg</img>
多项选择题：下面哪些选项正确的描述了这个图的部分特征：
A.	A red and yellow train is loading passengers at the station.
B.	a red and yellow building sits beside a running train
C.	red and yellow train stopped at a station
D.	A train is stopped at the train station.
答："""
response, history = model.chat(tokenizer, query=query, history=history)
print(response)
inputs = tokenizer(query, return_tensors='pt')
inputs = inputs.to(model.device)
pred = model.generate(**inputs)
response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=False)
print(response)

D. A train is stopped at the train station.
Picture 2:<img>./s0004800.jpg</img>
多项选择题：下面哪些选项正确的描述了这个图的部分特征：
A.	A red and yellow train is loading passengers at the station.
B.	a red and yellow building sits beside a running train
C.	red and yellow train stopped at a station
D.	A train is stopped at the train station.
答：C. red and yellow train stopped at a station
<|endoftext|>


In [4]:
history

[('Picture 1:<img>./s0004801.jpg</img>\n多项选择题：下面哪些选项正确的描述了这个图的部分特征：\nA.\tA chocolate cake with lots of candles in it.\nB.\tSomeone pinches a bite off of a homemade chocolate cake.\nC.\ta candy with some chocolate cake on top of it\n答：AB\n\nPicture 2:<img>./s0004800.jpg</img>\n多项选择题：下面哪些选项正确的描述了这个图的部分特征：\nA.\tA red and yellow train is loading passengers at the station.\nB.\ta red and yellow building sits beside a running train\nC.\tred and yellow train stopped at a station\nD.\tA train is stopped at the train station.\n答：',
  'C. red and yellow train stopped at a station')]

In [10]:
# 定义你感兴趣的 tokens 列表
tokens_of_interest = [str(chr(ord("A") + index)) for index in range(len(texts)) ]

# 将 tokens 转换为对应的 token IDs
token_ids_of_interest = [tokenizer(token,add_special_tokens=False).input_ids[0] for token in tokens_of_interest]

# 从 logits 中提取特定 token IDs 的 logits
logits_of_interest = predicted_token_id[:, token_ids_of_interest]

# 找到概率最高的 token
max_logits, max_indices = torch.max(logits_of_interest, dim=1)
predicted_token_id = token_ids_of_interest[max_indices.item()]

# 将 token ID 转换回 token
predicted_token = tokenizer.decode(predicted_token_id)

print(predicted_token)


B


In [1]:
import os

folder_path = "./data/test"
files = os.listdir(folder_path)

total = 0
counter = 0

temp_text_all = []
temp_image_all = []
texts_image_index = []

# 遍历文件夹中的每个文件
for file in files:
    # 确保文件是以'.json'为扩展名的JSON文件
    if file.endswith(".txt"):
        # 构建完整的文件路径
        file_path = os.path.join(folder_path, file)
        # 打开文件并读取JSON数据
        with open(file_path, "r", encoding="utf-8") as txt_file:
            captions = [x.strip() for x in txt_file.readlines()]

In [2]:
captions

['A young girl wades out into the water wearing safety floatation devices on her arms.',
 'The baby walks on the gold and sandy-colored beach with water splashing behind him.',
 'A girl at the shore of a beach with a mountain in the distance.',
 'Little girl in arm floaties exploring the coast line.',
 'A child playing in the ocean.']

In [4]:
from qwen_generation_utils import (
    HistoryType,
    make_context,
    decode_tokens,
    get_stop_words_ids,
    StopWordsLogitsProcessor,
)
sentences_batch = ['A young girl wades out into the water wearing safety floatation devices on her arms.',
 'The baby walks on the gold and sandy-colored beach with water splashing behind him.']
input_ids = []
max_length = 0
for j, sentence in enumerate(sentences_batch):
    query = tokenizer.from_list_format([
        {'text': f"""高等学校或研究机关中指导他人学习、进修、或撰写学术论文的教师或科研人员。\n上面这句话用一个中文词来表达是：导师。\n"""},
        {'text': f"""{sentence}\n上面这句话用一个中文词来表达是："""},
    ])
    raw_text, context_tokens = make_context(
        tokenizer,
        query,
        history=None,
        system="You are a helpful assistant.",
        max_window_size=model.generation_config.max_window_size,
        chat_format=model.generation_config.chat_format,
    )
    print(raw_text)
    # input_id = self.tokenizer(sentence).input_ids
    input_ids.append(context_tokens)
    max_length = max(max_length, len(context_tokens))
# max_length += 10
padding_lengths = []

for j in range(len(input_ids)):
    padding_lengths.append(max_length - len(input_ids[j]))
    input_ids[j] += [tokenizer.eod_id] * (max_length - len(input_ids[j]))
# target += [IGNORE_TOKEN_ID] * (max_len - len(target))
    # input_ids.append(input_id[:max_length])

# batch = self.tokenizer.batch_encode_plus(
#     sentences_batch,
#     return_tensors="pt",
#     padding=True,
#     max_length=max_length,
#     truncation=max_length is not None,
# )
new_input_ids = torch.tensor(input_ids, dtype=torch.int).to('cuda')
outputs = []
with torch.no_grad():
    attention_mask = new_input_ids.ne(tokenizer.eod_id)
    hidden_states = model(
        input_ids= new_input_ids,attention_mask = attention_mask,output_hidden_states=True, return_dict=True
    ).hidden_states
    last_true_indices = torch.max(attention_mask.int().cumsum(dim=1), dim=1).indices
    outputs = [hidden_states[-1][i, index, :] for i, index in enumerate(last_true_indices)]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
高等学校或研究机关中指导他人学习、进修、或撰写学术论文的教师或科研人员。
上面这句话用一个中文词来表达是：导师。
A young girl wades out into the water wearing safety floatation devices on her arms.
上面这句话用一个中文词来表达是：<|im_end|>
<|im_start|>assistant

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
高等学校或研究机关中指导他人学习、进修、或撰写学术论文的教师或科研人员。
上面这句话用一个中文词来表达是：导师。
The baby walks on the gold and sandy-colored beach with water splashing behind him.
上面这句话用一个中文词来表达是：<|im_end|>
<|im_start|>assistant



In [5]:
sentences_batch = ['A young girl wades out into the water wearing safety floatation devices on her arms.',
 'The baby walks on the gold and sandy-colored beach with water splashing behind him.']

single_outputs = []

for j, sentence in enumerate(sentences_batch):
    query = tokenizer.from_list_format([
        {'text': f"""高等学校或研究机关中指导他人学习、进修、或撰写学术论文的教师或科研人员。\n上面这句话用一个中文词来表达是：导师。\n"""},
        {'text': f"""{sentence}\n上面这句话用一个中文词来表达是："""},
    ])
    raw_text, context_tokens = make_context(
        tokenizer,
        query,
        history=None,
        system="You are a helpful assistant.",
        max_window_size=model.generation_config.max_window_size,
        chat_format=model.generation_config.chat_format,
    )
    # print(raw_text)
    # input_id = self.tokenizer(sentence).input_ids

    # input_ids.append(input_id[:max_length])

# batch = self.tokenizer.batch_encode_plus(
#     sentences_batch,
#     return_tensors="pt",
#     padding=True,
#     max_length=max_length,
#     truncation=max_length is not None,
# )
    input_ids = torch.tensor([context_tokens], dtype=torch.int).to('cuda')
    with torch.no_grad():
        hidden_states = model(
            input_ids= input_ids,attention_mask = input_ids.ne(tokenizer.eod_id),output_hidden_states=True, return_dict=True
        ).hidden_states
        output = hidden_states[-1][:, -1, :]
    single_outputs.append(output)

In [6]:
import torch.nn.functional as F

similarity_func = lambda s1, s2: torch.nan_to_num(
    F.cosine_similarity(torch.nan_to_num(s1), torch.nan_to_num(s2), dim=-1)
)

In [7]:
similarity_func(single_outputs[0],outputs[0]), similarity_func(single_outputs[1],outputs[1])

(tensor([0.9961], device='cuda:0', dtype=torch.bfloat16),
 tensor([1.], device='cuda:0', dtype=torch.bfloat16))